In [1]:
import sys
sys.path.append('./yolo')
from detection_fusion import EnsembleModel
import numpy as np
import matplotlib.pyplot as plt
from rcnn.dataset import ChestCocoDetection
import matplotlib.patches as patches
from torch.utils.data import DataLoader
import tqdm
import torch

In [2]:
from yolo.utils.general import xywh2xyxy, scale_coords, box_iou
from yolo.utils.metrics import ap_per_class
import torch
from rcnn.model import ChestRCNN
from yolo.yolo import Model
from yolo.utils.datasets import LoadImages
from yolo.utils.general import check_img_size

In [3]:
def add_bounding_boxes(target, ax):
    boxes = target['boxes']
    for box in boxes:
        mp_box = patches.Rectangle((box[0], box[1]), box[2] - box[0], box[3] - box[1], edgecolor="r", facecolor='none')
        ax.add_patch(mp_box)

def show_samples_for(test, train):
    fig, axs = plt.subplots(1, 2, figsize=(10,8))
    #fig.suptitle(f'Study: {samples["id"].iloc[i]}')
    for im_i, (img, data) in enumerate([test, train]):
        ax = axs[im_i] if isinstance(axs, np.ndarray) else axs
        ax.set_title(f'Instance / Image Nr. {im_i + 1} / {2}')
        #dcm = pydicom.dcmread(image_path)
        plt.figure()
        ax.imshow(img.permute(1,2,0))#,cmap=plt.cm.bone)
        add_bounding_boxes(data, ax)
    fig.savefig("combo.jpg")

In [4]:
test = ChestCocoDetection(root="F:\\aml-project\data\\siim-covid19-detection", ann_file="F:\\aml-project\\data\\siim-covid19-detection\\test.json", training=False, image_size=512, detection_fusion=True)
idx = np.random.randint(len(test))
test_img = test.__getitem__(idx)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [13]:
YOLO_FINAL_MODEL_PATH = "./yolo/models_final/yolov5_epoch_50.pt"
FASTER_RCNN_FINAL_MODEL_PATH = "./rcnn/models/fasterrcnn_epoch_23.pt"
RESNET_BACKBONE_PATH = "./resnet/models/resnext101_32x8d_epoch_35.pt"

yolov5_weights = torch.load(YOLO_FINAL_MODEL_PATH)
fasterrcnn_r101_weights = torch.load(FASTER_RCNN_FINAL_MODEL_PATH)

yolo = Model(cfg="./yolo/yolo5l.yaml",ch=3,nc=1)
yolo.load_state_dict(yolov5_weights, strict=False) 

fasterRcnn = ChestRCNN(RESNET_BACKBONE_PATH)
fasterRcnn.load_state_dict(fasterrcnn_r101_weights)
ensemble = EnsembleModel(fasterRcnn=fasterRcnn, yolo=yolo)

In [ ]:
results, boxes, scores, pred_labels, yolo_box, yolo_scores, yolo_labels, fcnn_box, fcnn_scores, fcnn_labels  = ensemble.detection_fusion(test_img[0].detach().clone().unsqueeze_(0), extended_output=True)

In [ ]:
from torchvision.ops import box_iou

In [ ]:
box_iou(torch.tensor(yolo_box),test_img[1]["boxes"])

In [ ]:
box_iou(torch.tensor(fcnn_box),test_img[1]["boxes"])

In [ ]:
show_samples_for(test_img, (test_img[0], {"boxes":boxes}))

### Evaluation of Ensemble method

In [14]:
def collate_fn(batch):
    imgs, targets, path = zip(*batch)
    imgs = torch.stack(imgs)
    return tuple((imgs, targets, path))

In [15]:
# need bs = 1 because model fusion cannot handle batches 
test = ChestCocoDetection(root="F:\\aml-project\data\\siim-covid19-detection", ann_file="F:\\aml-project\\data\\siim-covid19-detection\\test.json", training=False, image_size=512, detection_fusion=True)
test_loader = DataLoader(test, batch_size=1, shuffle=False, pin_memory=False, num_workers=0, collate_fn=collate_fn)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [16]:
jdict, stats, ap, ap_class = [], [], [], []
stats_yolo, stats_fcnn = [], []
iouv = torch.linspace(0.5, 0.55, 1) # iou vector for mAP@0.5:0.95
niou = iouv.numel()
general_test_results = []
for i, (images, targets, path) in enumerate(tqdm.tqdm(test_loader)):
    nb, _, height, width = images.shape
    assert nb == 1

    # Load image with Yolo transformations since we use the FCNN data loader per default
    path = '.\..\\data\\siim-covid19-detection\\' + path[0]
    stride = int(yolo.stride.max())  # model stride
    img_size = check_img_size(512, s=stride)  # check img_size
    yolo_img = LoadImages(path,img_size=img_size,stride=stride).__iter__().__next__()[1]
    yolo_img = torch.from_numpy(yolo_img).float()
    yolo_img /= 255.0
    if yolo_img.ndimension() == 3:
         yolo_img = yolo_img.unsqueeze(0)

    _, boxes, scores, pred_labels, yolo_box, yolo_scores, yolo_labels, fcnn_box, fcnn_scores, fcnn_labels = ensemble.detection_fusion(images,yolo_img, extended_output=True)
    boxes = torch.tensor(boxes)
    yolo_box = torch.tensor(yolo_box)
    fcnn_box = torch.tensor(fcnn_box)
    gt_boxes = targets[0]['boxes']
    labels = targets[0]['labels']
    nl = len(labels)
    tcls = labels[:].tolist() if nl else []  # target class

    if len(boxes) == 0:
        if nl:
            stats.append((torch.zeros(0, niou, dtype=torch.bool), torch.Tensor(), torch.Tensor(), tcls))
            stats_yolo.append((torch.zeros(0, niou, dtype=torch.bool), torch.Tensor(), torch.Tensor(), tcls))
            stats_fcnn.append((torch.zeros(0, niou, dtype=torch.bool), torch.Tensor(), torch.Tensor(), tcls))

        continue

    correct = torch.zeros(len(boxes), niou, dtype=torch.bool)
    correct_yolo = torch.zeros(len(yolo_box), niou, dtype=torch.bool)
    correct_fcnn = torch.zeros(len(fcnn_box), niou, dtype=torch.bool)
    
    if nl:
        detected = []
        detected_yolo = []
        detected_fcnn = []
        tcls_tensor = labels[:]

        # Per target class
        for cls in torch.tensor([1]):
            ti = (cls == tcls_tensor).nonzero(as_tuple=False).view(-1)
            pi = (cls == torch.tensor(pred_labels)).nonzero(as_tuple=False).view(-1)
            pi_yolo = (cls == torch.tensor(yolo_labels)).nonzero(as_tuple=False).view(-1)
            pi_fcnn = (cls == torch.tensor(fcnn_labels)).nonzero(as_tuple=False).view(-1)

            # Search for detections
            if pi.shape[0]:        
                ious, i = box_iou(boxes[pi], gt_boxes[ti]).max(1)  # best ious, indices
                # Append detections - Ensemble
                detected_set = set()
                for j in (ious > iouv[0]).nonzero(as_tuple=False):
                    d = ti[i[j]]  
                    if d.item() not in detected_set:
                        detected_set.add(d.item())
                        detected.append(d)
                        correct[pi[j]] = ious[j] > iouv  
                        if len(detected) == nl: 
                            break
                            
            if pi_yolo.shape[0]:
                ious_yolo, i_yolo = box_iou(yolo_box[pi_yolo], gt_boxes[ti]).max(1)
                # Yolo
                detected_set_yolo = set()
                for j in (ious_yolo > iouv[0]).nonzero(as_tuple=False):
                    d = ti[i_yolo[j]]  
                    if d.item() not in detected_set_yolo:
                        detected_set_yolo.add(d.item())
                        detected_yolo.append(d)
                        correct_yolo[pi_yolo[j]] = ious_yolo[j] > iouv  
                        if len(detected_yolo) == nl: 
                            break
                            
            if pi_fcnn.shape[0]:
                ious_fcnn, i_fcnn = box_iou(fcnn_box[pi_fcnn], gt_boxes[ti]).max(1)
                # Faster R-CNN
                detected_set_fcnn = set()
                for j in (ious_fcnn > iouv[0]).nonzero(as_tuple=False):
                    d = ti[i_fcnn[j]]  
                    if d.item() not in detected_set_fcnn:
                        detected_set_fcnn.add(d.item())
                        detected_fcnn.append(d)
                        correct_fcnn[pi_fcnn[j]] = ious_fcnn[j] > iouv  
                        if len(detected_fcnn) == nl: 
                            break



        # Append statistics (correct, conf, pcls, tcls)
        #print((correct.cpu(), torch.tensor(scores),torch.tensor(pred_labels), tcls))
        stats.append((correct.cpu(), torch.tensor(scores),torch.tensor(pred_labels), tcls))
        stats_yolo.append((correct_yolo.cpu(), torch.tensor(yolo_scores),torch.tensor(yolo_labels), tcls))
        stats_fcnn.append((correct_fcnn.cpu(), torch.tensor(fcnn_scores),torch.tensor(fcnn_labels), tcls))

for statistic in [stats, stats_yolo, stats_fcnn]:
    statistic = [np.concatenate(x, 0) for x in zip(*statistic)] 
    if len(statistic) and statistic[0].any():
        p, r, ap, f1, ap_class = ap_per_class(*statistic)
        ap50, ap = ap[:, 0], ap.mean(1) 
        mp, mr, map50, m = p.mean(), r.mean(), ap50.mean(), ap.mean()
        nt = np.bincount(statistic[3].astype(np.int64), minlength=1)

        general_test_results.append({
        "precision": p,
        "recall": r,
        "ap": ap,
        "f1": f1,
        "ap_class": ap_class,
        "ap": ap,
        "ap50": ap50,
        "mp": mp,
        "mr": mr,
        "map50": map50,
        "map": m
    })

    else:
        nt = torch.zeros(1)


100%|██████████| 859/859 [57:16<00:00,  4.00s/it]


In [17]:
general_test_results

[{'precision': array([    0.58684]),
  'recall': array([    0.52829]),
  'ap': array([    0.44636]),
  'f1': array([    0.55603]),
  'ap_class': array([1]),
  'ap50': array([    0.44636]),
  'mp': 0.5868425677325015,
  'mr': 0.5282898919262555,
  'map50': 0.4463627434092143,
  'map': 0.4463627434092143},
 {'precision': array([    0.52632]),
  'recall': array([    0.44501]),
  'ap': array([    0.33371]),
  'f1': array([    0.48226]),
  'ap_class': array([1]),
  'ap50': array([    0.33371]),
  'mp': 0.5263154357193186,
  'mr': 0.44500953591862685,
  'map50': 0.33371392387437626,
  'map': 0.33371392387437626},
 {'precision': array([    0.60309]),
  'recall': array([    0.50328]),
  'ap': array([    0.40929]),
  'f1': array([    0.54868]),
  'ap_class': array([1]),
  'ap50': array([    0.40929]),
  'mp': 0.6030946457346565,
  'mr': 0.5032776105936924,
  'map50': 0.40928837471026075,
  'map': 0.40928837471026075}]

In [10]:
general_test_results[0]

{'precision': array([    0.61747]),
 'recall': array([    0.50985]),
 'ap': array([    0.45336]),
 'f1': array([    0.55853]),
 'ap_class': array([1]),
 'ap50': array([    0.45336]),
 'mp': 0.6174727093486299,
 'mr': 0.5098537825810553,
 'map50': 0.45336404278151443,
 'map': 0.45336404278151443}

In [11]:
general_test_results[1]

{'precision': array([    0.47968]),
 'recall': array([    0.45709]),
 'ap': array([    0.34416]),
 'f1': array([    0.46811]),
 'ap_class': array([1]),
 'ap50': array([    0.34416]),
 'mp': 0.47968153556340803,
 'mr': 0.4570883661792753,
 'map50': 0.34416235531440953,
 'map': 0.34416235531440953}

In [12]:
general_test_results[2]

{'precision': array([    0.60309]),
 'recall': array([    0.50328]),
 'ap': array([    0.40929]),
 'f1': array([    0.54868]),
 'ap_class': array([1]),
 'ap50': array([    0.40929]),
 'mp': 0.6030946457346565,
 'mr': 0.5032776105936924,
 'map50': 0.40928837471026075,
 'map': 0.40928837471026075}